In [62]:
application = 'Watercare'

In [63]:
%matplotlib inline 

In [64]:
import pathlib

In [65]:
from matplotlib import pyplot as plt 

In [66]:
import pandas as pd

In [67]:
import numpy as np

In [68]:
HOME = pathlib.Path.home()

In [69]:
varname = 'RAIN_BC'

In [70]:
stat = 'min'

In [71]:
ipath = HOME.joinpath(f"research/Smart_Ideas/outputs/targets/application_cases/{application}/SEASONAL/{varname}")

In [72]:
data = pd.read_csv(ipath.joinpath(f"Seasonal_{varname}_{stat}_mean.csv"), index_col=0, parse_dates=True)

In [73]:
clim = data.loc['1981':'2010',:]

In [74]:
clim.quantile(q=[1/3, 2/3])

,Hunuas,Waitakere
0.333333,3.522628,3.725182
0.666667,4.894695,5.099092


### now calculates the categories 

### function to make terciles given a Pandas Series

In [75]:
def make_terciles(ts_series, climyears=['1981','2010'], num_quantiles=3): 

    if type(climyears[0] != str) or type(climyears[0] != str): 
        climyears = list(map(str, climyears))
    
    quantiles_dict = {}

    quant_values = np.linspace(0, 1, num_quantiles + 1, endpoint=True)
    
    quant_values = quant_values[1:-1]
    
    col_labs = [f"Q{int(x)}" for x in (quant_values*100)]
    
    ts_series_cat = []

    quantiles_list = []

    for month in range(1, 13):

        ts_series_m = ts_series[ts_series.index.month == month]

        clim = ts_series_m.loc[climyears[0]:climyears[1],]

        quantiles = [clim.quantile(q=q) for q in quant_values.tolist()]

        quantiles_list.append(quantiles.copy())

        quantiles.insert(0, -np.inf)

        quantiles.append(np.inf)

        ts_series_m_cats = pd.cut(ts_series_m, quantiles, labels=list(range(1, num_quantiles + 1)))

        ts_series_cat.append(ts_series_m_cats)

        del(quantiles)

    ts_series_cat = pd.concat(ts_series_cat, axis=0)

    ts_series_cat = ts_series_cat.sort_index()
    
    return ts_series_cat

def demean(x, climyears=['1981','2010']): 
    
    if type(climyears[0] != str) or type(climyears[0] != str): 
        climyears = list(map(str, climyears))
    
    return x - x.loc[climyears[0]:climyears[1],:].mean()

In [76]:
num_quantiles = 3

In [77]:
terciles_categories = data.apply(make_terciles, **{'climyears':[1981, 2010], 'num_quantiles':num_quantiles})

In [78]:
anoms = data.groupby(data.index.month).apply(demean, **{'climyears':[1981, 2010]})

In [79]:
df_all = pd.concat([terciles_categories, anoms], axis=1)

In [80]:
df_all.head()

,Hunuas,Waitakere,Hunuas,Waitakere
time,,,,
1979-03-31,3,3,1.849094,2.195274
1979-04-30,3,3,2.975399,3.564075
1979-05-31,3,3,0.831179,2.332098
1979-06-30,3,3,0.695597,2.404820
1979-07-31,3,3,2.061085,3.218922


In [81]:
columns = ['Hunas_cat3', 'Waitakere_cat3', 'Hunas_anomalies', 'Waitakere_anomalies']

In [82]:
df_all.columns = columns

In [83]:
df_all.head()

,Hunas_cat3,Waitakere_cat3,Hunas_anomalies,Waitakere_anomalies
time,,,,
1979-03-31,3,3,1.849094,2.195274
1979-04-30,3,3,2.975399,3.564075
1979-05-31,3,3,0.831179,2.332098
1979-06-30,3,3,0.695597,2.404820
1979-07-31,3,3,2.061085,3.218922


### the categories are now encoded as a categorical variable

In [84]:
df_all.Hunas_cat3.astype(np.int)

time
1979-03-31    3
1979-04-30    3
1979-05-31    3
1979-06-30    3
1979-07-31    3
             ..
2019-08-31    1
2019-09-30    3
2019-10-31    3
2019-11-30    2
2019-12-31    1
Name: Hunas_cat3, Length: 490, dtype: int64

### saves to disk 

In [85]:
df_all.to_csv(ipath.joinpath(f"Seasonal_{varname}_{stat}_anomalies_and_Q{num_quantiles}_categories.csv"))